In [25]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [26]:
# Load the trained model, scaler, and label encoder,one-hot encoder
model = load_model('churn_model.h5')

with open('scalar.pkl', 'rb') as f:
    scalar = pickle.load(f)

with open('le.pkl', 'rb') as f:
    le = pickle.load(f)

with open('ohe_ndarray.pkl', 'rb') as f:
    ohe_ndarray = pickle.load(f)

In [58]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000

}

In [59]:
# creating a DataFrame from input data
input_df= pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [60]:
#label encoder
input_df["Gender"]=le.transform(input_df['Gender'])
input_df = pd.concat((input_df.drop("Gender",axis=1), input_df["Gender"]), axis=1)

In [61]:
# One-hot encode 'Geography'
geo_encoded = ohe_ndarray.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_ndarray.get_feature_names_out(['Geography']))
geo_encoded_df
input_df=pd.concat((input_df.drop("Geography",axis=1),geo_encoded_df), axis=1)
input_df

c:\personal\Churn_ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,50000,1,1.0,0.0,0.0


In [62]:
# Ensure input_df columns are in the correct order
input_df = input_df[scalar.feature_names_in_]

# Then scale
input_df_scaled = scalar.transform(input_df)


In [69]:
input_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [65]:
prediction=model.predict(input_df_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step


array([[0.03356747]], dtype=float32)

In [ ]:
if prediction[0][0] > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


: 